In [2]:
from google.colab import files
uploaded = files.upload()

Saving GlobalLandTemperaturesByCountry.csv to GlobalLandTemperaturesByCountry.csv


In [15]:
# Step 0: Install and import necessary libraries
!pip install plotly pandas openpyxl

import pandas as pd
import plotly.express as px

# Step 1: Load the dataset
df = pd.read_csv('GlobalLandTemperaturesByCountry.csv')
df = df.dropna(subset=['AverageTemperature'])  # Remove rows with missing temperature values
df['dt'] = pd.to_datetime(df['dt'])  # Convert date column to datetime format
df['year'] = df['dt'].dt.year       # Extract year from date

# Step 2: Filter data from 1990 to 2020 for average temperature visualization
df_recent = df[(df['year'] >= 1990) & (df['year'] <= 2020)]

# Step 3: Calculate average temperature for each country over the selected period
df_avg = df_recent.groupby('Country')['AverageTemperature'].mean().reset_index()
df_avg.columns = ['Country', 'AvgTemp']

# Step 4: Define two historical time periods for trend analysis
df_early = df[(df['year'] >= 1970) & (df['year'] <= 1989)].groupby('Country')['AverageTemperature'].mean().reset_index()
df_early.columns = ['Country', 'TempEarly']

df_late = df[(df['year'] >= 2000) & (df['year'] <= 2020)].groupby('Country')['AverageTemperature'].mean().reset_index()
df_late.columns = ['Country', 'TempLate']

# Ensure both time periods include the same countries
common_countries = set(df_early['Country']).intersection(set(df_late['Country']))
df_early = df_early[df_early['Country'].isin(common_countries)]
df_late = df_late[df_late['Country'].isin(common_countries)]

# Step 5: Merge early and late period data, calculate temperature change
df_trend = pd.merge(df_early, df_late, on='Country')
df_trend['TempChange'] = df_trend['TempLate'] - df_trend['TempEarly']

# Merge average temperature and temperature change into one dataframe
df_merge = pd.merge(df_avg, df_trend[['Country', 'TempChange']], on='Country', how='left')

# Step 6A: Create Choropleth Map A – average temperature visualization
fig_temp = px.choropleth(
    df_merge,
    locations='Country',
    locationmode='country names',
    color='AvgTemp',
    color_continuous_scale='Inferno',
    title='🌡️ How Hot Is Too Hot? Average Land Temperature by Country (1990–2020)',
    hover_name='Country',
    hover_data={
        'AvgTemp': ':.2f',
        'TempChange': ':.2f'
    }
)
fig_temp.update_layout(
    title=dict(
        text='🌡️ How Hot Is Too Hot? <br><sup>This map shows average land temperature from 1990–2020. '
             'Countries with the fastest warming trend are highlighted. Temperature is not just data—it’s lived experience.</sup>',
        x=0.5,
        xanchor='center'
    )
)

# Step 6B: Create Choropleth Map B – highlight top 10 fastest warming countries
top10_countries = df_trend.sort_values('TempChange', ascending=False).head(10)['Country'].tolist()
df_merge['Highlight'] = df_merge['Country'].apply(lambda x: 'Top 10 Rising' if x in top10_countries else 'Others')

fig_highlight = px.choropleth(
    df_merge,
    locations='Country',
    locationmode='country names',
    color='Highlight',
    color_discrete_map={
        'Top 10 Rising': 'crimson',
        'Others': 'lightgray'
    },
    title='🔥 Top 10 Fastest Warming Countries Highlighted (1970–2020)',
    hover_name='Country',
    hover_data={'TempChange': ':.2f'}
)

# Display both visualizations
fig_temp.show()
fig_highlight.show()

<ipython-input-15-b99e464edfd2>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-15-b99e464edfd2>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

